In [ ]:
start = time();

In [ ]:
using CUDA
using Printf
using Statistics
using Juliana

In [ ]:
CUDA.devices()

# Configuration

In [ ]:
path_to_phantom = "/data/user/bellotti_r/data_special_cases/renate";
target_dose = 60.f0; # Gy RBE

fiona_standalone_bin_path = "/data/user/bellotti_r/semester_project_planning_metrics/src/pyftpp/bin"
fiona_jar_path = "$fiona_standalone_bin_path/ch.psi.ftpp.standalone.planner-1.0.10.jar";
output_dir = "../output/demo_notebook"

mkpath(output_dir);

# Load data

In [ ]:
ct, structures = Juliana.load_dicom_directory(
    path_to_phantom;
    structure_names=["PTV_total_2mm", "BrainStem_Surface"],
);
target = structures["PTV_total_2mm"];

constraints = Juliana.parse_oar_constraints_file(
    "$(path_to_phantom)/constraints.csv",
    target_dose,
    structures,
)
prescriptions = Juliana.Prescriptions([("PTV_total_2mm", target_dose)], constraints);

In [ ]:
# This is the only TPS-specific line in the entire notebook!
tps = Juliana.load_juliana_tps(fiona_standalone_bin_path, 0);

# Optimise

In [ ]:
field = Juliana.FieldDefinition(
    "F0",                                             # field label
    0,                                                # field ID
    37.f0,                                            # gantry angle
    63.f0,                                            # couch angle
    20.f0,                                            # nozzle extraction
    mean(structures["PTV_total_2mm"].points, dims=1), # iso-center
);

In [ ]:
@time plan = Juliana.place_spots(
    tps,
    ct,
    [field],
    [target],
);

In [ ]:
@time plan_optimised = Juliana.optimise_juliana(tps, ct, structures, prescriptions, plan);

# Calculate the dose distribution

In [ ]:
@time dose = Juliana.calculate_dose(
    tps,
    0.35f0, # dose grid resolution
    ct,
    plan_optimised,
);

In [ ]:
stop = time();

In [ ]:
@printf "Time to run the demo notebook: %.2fs" stop - start

# Plot the dose distribution

In [ ]:
dose_to_plot = copy(dose.data)
dose_to_plot[ct.data .<= -100] .= 0.f0;

In [ ]:
fig, axes = Juliana.plot_distributions(
    ct,
    (("", dose_to_plot),),
    (("PTV", (target, "red")), ),
    145, 360,
    100, 330,
    100,
    target_dose,
    Juliana.build_colorscheme(),
)

# Export to DICOM

In [ ]:
new_patient_ID = "juliana_example_HN"
study_instance_uid = Juliana.get_study_instance_uid(new_patient_ID);

In [ ]:
Juliana.dicom_export_to_directory(
    ct,
    structures,
    output_dir,
    study_instance_uid,
    new_patient_ID,
    Dict{String, Juliana.ScalarGrid}(
        "juliana" => dose,
    ),
);